In [26]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

from torchsampler import ImbalancedDatasetSampler
from net import Net

In [27]:
train_dir = './train_images'    # folder containing training images
test_dir = './test_images'    # folder containing test images

valid_size = 0.2   # proportion of validation set (80% train, 20% validation)
batch_size = 32  


In [29]:
def create_dataloader(train_dataset, test_dataset, validation_size=0.2, batch_size=32, unbalanced=False):
    # Define randomly the indices of examples to use for training and for validation
    num_train = len(train_dataset)
    indices_train = list(range(num_train))
    np.random.shuffle(indices_train)
    split_tv = int(np.floor(validation_size * num_train))
    
    train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

    if not unbalanced:
        # Define two "samplers" that will randomly pick examples from the training and validation set
        train_sampler = SubsetRandomSampler(train_new_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)
    else:
        # Define two "samplers" that will randomly pick examples from the training and validation set in an unbalanced way
        train_sampler = ImbalancedDatasetSampler(train_dataset, train_new_idx)
        valid_sampler = ImbalancedDatasetSampler(train_dataset, valid_idx)
        
    # Dataloaders (take care of loading the data from disk, batch by batch, during training)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
    valid_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    return train_loader, valid_loader, test_loader

In [30]:
transform = transforms.Compose(
    [transforms.Grayscale(),   # transforms to gray-scale (1 input channel)
     transforms.ToTensor(),    # transforms to Torch tensor (needed for PyTorch)
     transforms.Normalize(mean=(0.5,),std=(0.5,))]) # subtracts mean (0.5) and devides by standard deviation (0.5) -> resulting values in (-1, +1)

In [31]:
# Define two pytorch datasets (train/test) 
train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# Define the train_data loader, validation_data loader, test_data loader
train_loader, valid_loader, test_loader = create_dataloader(train_data, test_data, 0.2, 32, unbalanced=False)

In the next step, we will determine the optimizer for our neural network, i.e the algorithm to adjust model's parameter

In [6]:
net = Net()
n_epochs = 32

optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [7]:
# Training 
running_loss =0
# loop over epochs: one epoch = one pass through the whole training dataset
for epoch in range(1, n_epochs+1):  
#   loop over iterations: one iteration = 1 batch of examples
    running_loss =0
    for data, target in train_loader: 
        optimizer.zero_grad() # zero the gradient buffers
        output = net(data)
        loss = criterion(output, target)
        running_loss +=loss
        loss.backward()
        optimizer.step() # Does the update
    print ('epoch: %d, running_loss: %5.7f' % (epoch,running_loss))  

epoch: 1, running_loss: 214.3142853
epoch: 2, running_loss: 62.5703545
epoch: 3, running_loss: 38.7535667
epoch: 4, running_loss: 29.8845310
epoch: 5, running_loss: 23.4209633
epoch: 6, running_loss: 19.9186306
epoch: 7, running_loss: 17.7311115
epoch: 8, running_loss: 14.0981884
epoch: 9, running_loss: 14.9567232
epoch: 10, running_loss: 13.0569525
epoch: 11, running_loss: 9.7059460
epoch: 12, running_loss: 10.8036327
epoch: 13, running_loss: 10.3880339
epoch: 14, running_loss: 9.7510948
epoch: 15, running_loss: 9.2061424
epoch: 16, running_loss: 9.8168344
epoch: 17, running_loss: 8.4004841
epoch: 18, running_loss: 8.3007460
epoch: 19, running_loss: 7.4232326
epoch: 20, running_loss: 8.7580853
epoch: 21, running_loss: 8.1304035
epoch: 22, running_loss: 7.0545006
epoch: 23, running_loss: 7.9373608
epoch: 24, running_loss: 6.8218036
epoch: 25, running_loss: 7.5354710
epoch: 26, running_loss: 5.8522930
epoch: 27, running_loss: 7.3025656
epoch: 28, running_loss: 6.3039107
epoch: 29, runni

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %5.6f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 94.611956 %


# Augmented

In [32]:
# Define the transformations for augmentation
transforms_train_augmented = transforms.Compose([
    transforms.Grayscale(), 
    transforms.RandomResizedCrop(36),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,),std=(0.5,))
])       

net_augmented = Net()

optimizer = optim.Adam(net_augmented.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


In [33]:
# Define pytorch augmented train datasets 
train_data_augmented = torchvision.datasets.ImageFolder(train_dir, transform=transforms_train_augmented)

# Define the train_data loader, validation_data loader, test_data loader
train_loader, valid_loader, test_loader = create_dataloader(train_data_augmented, test_data, 0.2, 32, unbalanced=False)

In [34]:

n_epochs = 32
# Training 
running_loss =0
# loop over epochs: one epoch = one pass through the whole training dataset
for epoch in range(1, n_epochs+1):  
#   loop over iterations: one iteration = 1 batch of examples
    running_loss =0
    for data, target in train_loader: 
        optimizer.zero_grad() # zero the gradient buffers
        output = net_augmented(data)
        loss = criterion(output, target)
        running_loss +=loss
        loss.backward()
        optimizer.step() # Does the update
    print ('epoch: %d, running_loss: %5.7f' % (epoch,running_loss))  

epoch: 1, running_loss: 769.6201172
epoch: 2, running_loss: 454.5734558
epoch: 3, running_loss: 362.4648743
epoch: 4, running_loss: 323.7208252
epoch: 5, running_loss: 290.5789185
epoch: 6, running_loss: 279.4784851
epoch: 7, running_loss: 266.4322815
epoch: 8, running_loss: 248.9024353
epoch: 9, running_loss: 243.9502106
epoch: 10, running_loss: 239.5317688


In [13]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net_augmented(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %5.6f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10.448348 %


# Unbalanced

In [14]:
# Define two pytorch datasets (train/test) 
train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# Define the train_data loader, validation_data loader, test_data loader
train_loader, valid_loader, test_loader = create_dataloader(train_data, test_data, 0.2, 32, unbalanced=True)

In [15]:
# Training 
running_loss =0
# loop over epochs: one epoch = one pass through the whole training dataset
for epoch in range(1, n_epochs+1):  
#   loop over iterations: one iteration = 1 batch of examples
    running_loss =0
    for data, target in train_loader: 
        optimizer.zero_grad() # zero the gradient buffers
        output = net(data)
        loss = criterion(output, target)
        running_loss +=loss
        loss.backward()
        optimizer.step() # Does the update
    print ('epoch: %d, running_loss: %5.7f' % (epoch,running_loss))

epoch: 1, running_loss: 10.3706675
epoch: 2, running_loss: 7.0060797
epoch: 3, running_loss: 7.4594674
epoch: 4, running_loss: 6.1392665
epoch: 5, running_loss: 7.0472698


KeyboardInterrupt: 

In [ ]:
classification_map = {"TP" : 0,
                      "FP" : 0,
                      "TN" : 0,
                      "FN" : 0}

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        
        for i in range(0,len(labels)):
            if predicted[i].item() == labels[i].item():
                if predicted[i].item() == 1:
                    classification_map["TP"] +=1
                else:
                    classification_map["TN"] +=1
            elif predicted[i].item() == 1:
                classification_map["FP"] +=1
            else: 
                classification_map["FN"] +=1

        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %5.6f %%' % (
    100 * correct / total))

In [ ]:
stats_map = {
            "Specificity" : float(classification_map["TN"]) / float(classification_map["TN"] + classification_map["FP"]),
            "Recall" : float(classification_map["TP"]) / float(classification_map["TP"] + classification_map["FN"]),
            "Precision" : float(classification_map["TP"]) / float(classification_map["TP"] + classification_map["FP"]),
            "Accuracy" : float(classification_map["TP"] + classification_map["TN"]) / float(classification_map["TP"] + classification_map["TN"] + classification_map["FP"] + classification_map["FN"])
        }
stats_map["F-score"] = 2.0 / float((1.0 / float(stats_map["Precision"])) + (1.0 / float(stats_map["Recall"])))

for key, value in stats_map.items():
    print(key, ": ", value)